<a href="https://colab.research.google.com/github/aSafarpoor/Seminar/blob/main/ilpc2022_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/MSc/codes/ilpc2022/

/content/drive/MyDrive/MSc/codes/ilpc2022


In [4]:
!pip install pykeen
# !pip install click
!pip install wandb 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 630 kB 7.4 MB/s 
     |████████████████████████████████| 38.1 MB 382 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 308 kB 68.9 MB/s 
     |████████████████████████████████| 81 kB 12.1 MB/s 
     |████████████████████████████████| 210 kB 77.6 MB/s 
     |████████████████████████████████| 256 kB 64.5 MB/s 
     |████████████████████████████████| 78 kB 9.6 MB/s 
     |████████████████████████████████| 112 kB 57.5 MB/s 
     |████████████████████████████████| 49 kB 8.8 MB/s 
     |████████████████████████████████| 146 kB 72.5 MB/s 
     |████████████████████████████████| 47 kB 5.1 MB/s 
  Created wheel for click-default-group: filename=click_default_group-1.2.2-py3-none-any.whl size=3385

In [4]:
import datetime 

In [5]:
"""The challenge's datasets."""
from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from typing_extensions import Literal

__all__ = [
    "InductiveLPDataset",
    "Size",
]

# HERE = Path(__file__).parent.resolve()
DATA = "data/"

Size = Literal["small/", "large/", "sp/"]


class InductiveLPDataset(DisjointInductivePathDataset):
    """An inductive link prediction dataset for the ILPC 2022 Challenge."""

    def __init__(self, size= "small/", **kwargs):
        """Initialize the inductive link prediction dataset.

        :param size: "small" or "large"
        :param kwargs: keyword arguments to forward to the base dataset class, cf. DisjointInductivePathDataset
        """

        super().__init__(
            transductive_training_path=size+ "train.txt",
            inductive_inference_path=size+ "inference.txt",
            inductive_validation_path=size+ "inference_validation.txt",
            inductive_testing_path=size+ "inference_test.txt",
            create_inverse_triples=True,
            eager=True,
            **kwargs
        )


In [21]:
"""Example workflow."""
import logging
from pathlib import Path

# import click
# import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


DATA = "data/"

# fix the seed for reproducibility
set_random_seed(42)


# for GNN layer reproducibility
# when running on a GPU, make sure to set up an env variable as advised in the doc:
# https://pytorch.org/docs/stable/generated/torch.use_deterministic_algorithms.html

# torch.use_deterministic_algorithms(True)

'''
click.Choice(["small", "large","sp"]),
default="small",
    show_default=True,
    help="The dataset to use",
)

    "--embedding-dim",
    help="The dimension of the entity embeddings",

    "--tokens",
    help="Number of tokens to use in NodePiece",

    "--margin",
    help="The margin value to use for the negative sampling self-adversarial loss.",

    "--num-negatives",
    help="The number of negative samples per positive.",

    "--wandb",
    help="Track results with Weights & Biases (requires `wandb` to be installed).",

    "--save",
     is_flag=True, help=f"Save the model in the {DATA} directory")

    "--gnn", 
    is_flag=True, help="Use the Inductive NodePiece model with GCN layers"

@more_click.log_level_option()
'''


def main(
    dataset= 'sp/',
    embedding_dim= 100,
    tokens= 5,
    learning_rate= 0.001,
    margin= 15.0,
    num_negatives= 4,
    batch_size= 256,
    epochs= 2,
    wandb= True,
    save= True,
    gnn= True,
    log_level= ""):


    """Train an inductive model with NodePiece representations and an optional GNN encoder."""
    # set appropriate log-level
    logging.basicConfig(level=log_level)

    # dataset loading
    dataset = InductiveLPDataset(size=dataset)
    loss = NSSALoss(margin=margin)

    # we have two baselines: InductiveNodePiece and InductiveNodePieceGNN
    # the GNN version uses a 2-layer CompGCN message passing encoder on the training / inference graphs
    # but feel free to create and attach your own GNN encoder via the gnn_encoder argument
    # and new inductive link prediction models in general
    model_cls = InductiveNodePieceGNN if gnn else InductiveNodePiece
    model = model_cls(
        embedding_dim=embedding_dim,
        triples_factory=dataset.transductive_training,
        inference_factory=dataset.inductive_inference,
        num_tokens=tokens,
        aggregation="mlp",
        loss=loss,
    ).to(resolve_device())
    optimizer = Adam(params=model.parameters(), lr=learning_rate)
    logging.info(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
    logging.info(f"Space occupied: {model.num_parameter_bytes} bytes")

    if wandb:
        tracker = WANDBResultTracker(
            project="inductive_ilp",  # put here your project and entity
            entity="pykeen",
            # config=click.get_current_context().params,
        )
        tracker.start_run()
    else:
        tracker = ConsoleResultTracker()

    # default training regime is negative sampling (SLCWA)
    # you can also use the 1-N regime with the LCWATrainingLoop
    # the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
    training_loop = SLCWATrainingLoop(
        triples_factory=dataset.transductive_training,
        model=model,
        optimizer=optimizer,
        result_tracker=tracker,
        negative_sampler_kwargs=dict(
            # affects training speed, the more - the better
            num_negs_per_pos=num_negatives
        ),
        mode=TRAINING,  # must be specified for the inductive setup
    )

    # specifying hits@k values: 1, 3, 5, 10, 100
    valid_evaluator = RankBasedEvaluator(
        mode=VALIDATION,
        metrics=["hits_at_k"]*5,
        metrics_kwargs=[dict(k=k) for k in (1, 3, 5, 10, 100)],
        add_defaults=True,
    )
    test_evaluator = RankBasedEvaluator(
        mode=TESTING,
        metrics=["hits_at_k"] * 5,
        metrics_kwargs=[dict(k=k) for k in (1, 3, 5, 10, 100)],
        add_defaults=True
    )

    # model training and eval on validation starts here
    training_loop.train(
        triples_factory=dataset.transductive_training,
        num_epochs=epochs,
        batch_size=batch_size,
        callbacks="evaluation",
        callback_kwargs=dict(
            evaluator=valid_evaluator,
            evaluation_triples=dataset.inductive_validation.mapped_triples,
            prefix="validation",
            frequency=1,
            additional_filter_triples=dataset.inductive_inference.mapped_triples,
            batch_size=batch_size,
        ),
    )

    # final eval on the test set
    result = test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.inductive_testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.inductive_inference.mapped_triples,
            dataset.inductive_validation.mapped_triples,
        ],
        batch_size=batch_size,
    )

    # extracting final metrics
    for metric, metric_label in [
        ("inverse_harmonic_mean_rank", "MRR"),
        *((f"hits_at_{k}", f"Hits@{k}") for k in (100, 10, 3, 1)),
        ("adjusted_arithmetic_mean_rank_index", "AMRI"),
    ]:
        logging.info(f"Test {metric_label:10}: {result.get_metric(name=metric):.5f}")

    # you can also log the final results to wandb if you want
    if wandb:
        tracker.log_metrics(
            metrics=result.to_flat_dict(),
            step=epochs + 1,
            prefix="test",
        )

    # saving the final model
    if save:
        current_time = datetime.datetime.now() 
        s = current_time.isoformat()
        print(s)
        torch.save(model, "result/model_"+s)

        





In [ ]:
main(dataset= 'data/small/',
    embedding_dim= 100,
    tokens= 5,
    learning_rate= 0.001,
    margin= 15.0,
    num_negatives= 4,
    batch_size= 256,
    epochs= 50,
    wandb= False,
    save= True,
    gnn= True,
    log_level= "Trace"
    )

sampling:   0%|          | 0.00/10.2k [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pykeen/nn/representation.py:372: UserWarning: Directly use Embedding.max_id instead of num_embeddings.
  warnings.warn(f"Directly use {self.__class__.__name__}.max_id instead of num_embeddings.")


sampling:   0%|          | 0.00/6.65k [00:00<?, ?it/s]

INFO:pykeen.triples.triples_factory:Creating inverse triples.


Training epochs on cuda:0:   0%|          | 0/50 [00:00<?, ?epoch/s]

...

Training batches on cuda:0:   0%|          | 0/614 [00:00<?, ?batch/s]

Evaluating on cuda:0:   0%|          | 0.00/2.91k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.55s seconds


Step: 7
Metric: validation.head.optimistic.hits_at_1 = 0.003782668500687758
Metric: validation.tail.optimistic.hits_at_1 = 0.031292984869325996
Metric: validation.both.optimistic.hits_at_1 = 0.017537826685006877
Metric: validation.head.realistic.hits_at_1 = 0.003782668500687758
Metric: validation.tail.realistic.hits_at_1 = 0.031292984869325996
Metric: validation.both.realistic.hits_at_1 = 0.017537826685006877
Metric: validation.head.pessimistic.hits_at_1 = 0.003782668500687758
Metric: validation.tail.pessimistic.hits_at_1 = 0.031292984869325996
Metric: validation.both.pessimistic.hits_at_1 = 0.017537826685006877
Metric: validation.head.optimistic.hits_at_3 = 0.015130674002751032
Metric: validation.tail.optimistic.hits_at_3 = 0.10213204951856947
Metric: validation.both.optimistic.hits_at_3 = 0.05863136176066025
Metric: validation.head.realistic.hits_at_3 = 0.015130674002751032
Metric: validation.tail.realistic.hits_at_3 = 0.10213204951856947
Metric: validation.both.realistic.hits_at_3 =

Training batches on cuda:0:   0%|          | 0/614 [00:00<?, ?batch/s]